<a href="https://colab.research.google.com/github/Hamza-Suleman/transiting-exoplanets/blob/main/Wrapper_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating a Wrapper Function

This notebook will follow the creation and testing of a wrapper function to analyse raw transit data using Python.

##Installing Dependencies

In [3]:
!pip install batman-package
!pip install astropy
!pip install transitleastsquares
from scipy.optimize import minimize
!python -c 'import batman;batman.test()'
import batman
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from transitleastsquares import transitleastsquares
from IPython.display import display, Math
!pip install -U setuptools setuptools_scm pep517
!pip install -U emcee
import emcee
import statistics


Starting tests...

Testing uniform limb darkening model...	
	test passed
Testing linear limb darkening model...	
	test passed
Testing quadratic limb darkening model...	
	test passed
Testing exponential limb darkening model...	
	test passed
Testing logarithmic limb darkening model...	
	test passed
Testing nonlinear limb darkening model...	
	test passed

Testing error tolerance...
	10.0 ppm passed
	1.0 ppm passed
	0.1 ppm passed
	0.01 ppm passed

Testing multithreading...
	test passed

Congratulations! all tests passed

  ____       ____
  )   \     /   (
   )_  \_V_/  _(
     )__   __(
        `-'


##Creating the Wrapper Function

This code will be the main body of the wrapper function, it will take various inputs and find the parameters of the transit and their errors.

In [61]:
def transit_analysis(data):

  manual_params = [3.719,4.78,0.085,10,90.5] #This is an empty list that will eventually populate with manaul input parameters.
  param_definitions = ["Time of Inferior Conjunction", "Orbital Period", "Planet Radius", "Semi-major Axis", "Orbital Inclination" ]

  #Importing raw data
  raw_data = ascii.read(data)
  raw_time = []
  raw_flux = []
  raw_flux_err = []
  for i in range(len(raw_data)):
    raw_time.append(raw_data[i][0])
    raw_flux.append(raw_data[i][1])
    raw_flux_err.append(raw_data[i][2])
  raw_time = np.array(raw_time)
  raw_flux = np.array(raw_flux)
  raw_flux_err = np.array(raw_flux_err)
  #plt.plot(raw_time,raw_flux)

  #A function to plot a transit from BATMAN parameters, and a list of time and flux values.
  def transit_plot(par,t,xaxis="Time",plottype="line",colour="b",transparency=None):   #creating a function to take a list parameter inputs and plot a lightcurve
    params = batman.TransitParams()
    params.t0 = par[0]                      #time of inferior conjunction
    params.per = par[1]                     #orbital period
    params.rp = par[2]                      #planet radius (in units of stellar radii)
    params.a = par[3]                       #semi-major axis (in units of stellar radii)
    params.inc = par[4]                     #orbital inclination (in degrees)
    params.ecc = 0                          #eccentricity
    params.w = 90                           #longitude of periastron (in degrees)
    params.u = [0.1, 0.3]                   #limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"          #limb darkening model
    m = batman.TransitModel(params, t)      #initializes model
    flux = m.light_curve(params)            #calculates light curve
    if plottype == "scatter":
      plt.scatter(t,flux,color=colour,alpha=transparency)
    else:
      plt.plot(t, flux,color=colour,alpha=transparency)
    plt.xlabel(xaxis)
    plt.ylabel("Relative flux")
    plt.show()

  #This function will allow the user to fit parameters manually to the data, inside the wrapper function.
  def manual_fitting(t,f,par):
    user_input = "no"
    while user_input.lower() == "no":
      print("\nTrying Parameters:")
      print(manual_params)
      #plt.figure(figsize=(30,10))
      plt.scatter(t,f)
      transit_plot(manual_params,t,colour="r",transparency=0.7)
      #plt.show()
      user_input = input("\nDo these parameters fit?\n")
      if user_input.lower() == "no":
        for i in range(len(manual_params)):
          manual_params[i] = float(input("Enter a value for " + param_definitions[i] + " Parameter: "))
    print("\nFinal Manual Parameters:")
    print(manual_params)

  #cleaning the manual parameters to be used later in MCMC analysis
  if manual_params[4] > 90:
    manual_params[4] = 90-(manual_params[4]-90)

  #manual_fitting(raw_time,raw_flux,manual_params)

  #MCMC Analysis (on parameters: inferior conjunction, planet radius, semi-major axis, inclination)
  def log_likelihood(par, t, flux, flux_error):
    params = batman.TransitParams()
    params.t0 = par[0]                      #time of inferior conjunction
    params.per = 4.78070560                 #orbital period
    params.rp = par[1]                      #planet radius (in units of stellar radii)
    params.a = par[2]                       #semi-major axis (in units of stellar radii)
    params.inc = par[3]                     #orbital inclination (in degrees)
    params.ecc = 0                          #eccentricity
    params.w = 90                           #longitude of periastron (in degrees)
    params.u = [0.1, 0.3]                   #limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic" 
    LC_model = batman.TransitModel(params, t)
    LC_flux = LC_model.light_curve(params)  #calulating lightcurve flux values from input parameters
    return -0.5 * np.nansum(((LC_flux - flux)**2)/flux_error**2) #return log likelihood
  
  def log_prior(par):
    #print(par)
    if par[0] > 0 and 0 < par[1] < 1 and par[2] > 0 and 0 < par[3] < 90: #these are limits we know to be true for the parameters that we have
        return 0.0
    #print("inf")
    return -np.inf

  def log_probability(par, t, flux, flux_error):
    lp = log_prior(par)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(par, t, flux, flux_error)

  def MCMC(par, t, flux, flux_error):
    pos = par + 1e-4 * np.random.randn(50, 4)
    nwalkers, ndim = pos.shape
    sampler = emcee.EnsembleSampler(
      nwalkers, ndim, log_probability, args=(t, flux, flux_error)
    )
  sampler.run_mcmc(pos, 5000, progress=True); 
  samples = sampler.get_chain()
  fig, axes = plt.subplots(4, figsize=(10, 7), sharex=True)
  labels = ["t0", "rp", "a","inc"]
  for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
  axes[-1].set_xlabel("step number");
  params = np.zeros((4,3))
  flat_samples = sampler.get_chain(discard=1000, thin=15, flat=True)
  print("These are the variables calculated by MCMC analysis:")
  for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.5f}_{{-{1:.5f}}}^{{{2:.5f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))
    params[i] = (mcmc[1], q[0], q[1])
  return flat_samples,params

  mcmc_input_params = [manual_params[0],manual_params[2],manual_params[3],manual_params[4]]
  mcmc_input_params = np.array(mcmc_input_params)
  mcmc_output = MCMC(mcmc_input_params,raw_time,raw_flux,raw_flux_err)
  flat_samples = mcmc_output[0]
  mcmc_params = mcmc_output[1]
  print("Output Parameters from MCMC Analysis")
  print(mcmc_params)

In [ ]:
transit_analysis("https://raw.githubusercontent.com/Hamza-Suleman/transiting-exoplanets/main/Transit_model_LC_data.txt")